<a href="https://colab.research.google.com/github/ben-wycliff/dl-final-exam/blob/main/torch_experiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/School/MSc Computer Science/Sem2 - (Ben Wycliff) - Year 1/Deep Learning/deep learning final exam/data.zip"

# Import Libraries

In [ ]:
import os
import torch
import torchvision
from torchvision.transforms import transforms

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np

# Prepare Data

In [ ]:
# Set the path to your dataset
train_dir = 'data/train'
test_dir = 'data/test'
val_dir = 'data/val'

# Set the input image dimensions
input_shape = (256, 256)

# Set the number of classes
num_classes = 6

# Define transformations for data normalization and grayscale conversion
data_transforms = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(input_shape),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])])

# Create datasets for training, validation, and testing
train_dataset = torchvision.datasets.ImageFolder(root=train_dir, transform=data_transforms)
val_dataset = torchvision.datasets.ImageFolder(root=val_dir, transform=data_transforms)
test_dataset = torchvision.datasets.ImageFolder(root=test_dir, transform=data_transforms)

# Create data loaders for training, validation, and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Model

In [ ]:
# Define the convolutional neural network model
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(32*32*128, 32),
    nn.ReLU(),
    nn.Linear(32, num_classes),
    nn.Softmax(dim=1)
)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up TensorBoard writer
log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

# Train

In [ ]:
train_loss_history = []
train_acc_history = []
valid_loss_history = []
valid_acc_history = []

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

best_valid_acc = 0.0
num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    train_running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Update training loss and accuracy values
        train_running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    # Calculate training loss and accuracy for the epoch
    train_epoch_loss = train_running_loss / len(train_loader)
    train_epoch_acc = train_correct / train_total

    # Append training loss and accuracy values to the history lists
    train_loss_history.append(train_epoch_loss)
    train_acc_history.append(train_epoch_acc)

    # Validation step
    model.eval()
    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            # Update validation loss and accuracy values
            valid_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()

    # Calculate validation loss and accuracy for the epoch
    valid_epoch_loss = valid_running_loss / len(val_loader)
    valid_epoch_acc = valid_correct / valid_total

    # Append validation loss and accuracy values to the history lists
    valid_loss_history.append(valid_epoch_loss)
    valid_acc_history.append(valid_epoch_acc)

    # Print the loss and accuracy for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_acc:.4f}, '
          f'Valid Loss: {valid_epoch_loss:.4f}, Valid Acc: {valid_epoch_acc:.4f}')

    # Check if the current model's validation accuracy is better than the previous best accuracy
    if valid_epoch_acc > best_valid_acc:
        best_valid_acc = valid_epoch_acc
        best_model_weights = model.state_dict()

print('Training finished.')

Epoch [1/40], Train Loss: 1.6497, Train Acc: 0.3859, Valid Loss: 1.5486, Valid Acc: 0.4778
Epoch [2/40], Train Loss: 1.5812, Train Acc: 0.4621, Valid Loss: 1.5511, Valid Acc: 0.4750
Epoch [3/40], Train Loss: 1.5577, Train Acc: 0.4855, Valid Loss: 1.5788, Valid Acc: 0.4472
Epoch [4/40], Train Loss: 1.5597, Train Acc: 0.4831, Valid Loss: 1.4932, Valid Acc: 0.5389
Epoch [5/40], Train Loss: 1.5110, Train Acc: 0.5326, Valid Loss: 1.4610, Valid Acc: 0.5722
Epoch [6/40], Train Loss: 1.5097, Train Acc: 0.5333, Valid Loss: 1.4602, Valid Acc: 0.5722
Epoch [7/40], Train Loss: 1.4698, Train Acc: 0.5730, Valid Loss: 1.5168, Valid Acc: 0.5194
Epoch [8/40], Train Loss: 1.4880, Train Acc: 0.5547, Valid Loss: 1.4306, Valid Acc: 0.6056
Epoch [9/40], Train Loss: 1.4654, Train Acc: 0.5769, Valid Loss: 1.4363, Valid Acc: 0.5972
Epoch [10/40], Train Loss: 1.4513, Train Acc: 0.5915, Valid Loss: 1.4282, Valid Acc: 0.6056
Epoch [11/40], Train Loss: 1.4364, Train Acc: 0.6066, Valid Loss: 1.4012, Valid Acc: 0.62

In [ ]:
# Evaluate the model on the validation set
model.eval()
val_predictions = []
val_labels = []
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        val_predictions.extend(predicted.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Convert prediction labels and true labels to numpy arrays
val_predictions = np.array(val_predictions)
val_labels = np.array(val_labels)

In [ ]:
# Print classification report and confusion matrix
print("Validation Classification Report:")
print(classification_report(val_labels, val_predictions))
print("Confusion Matrix:")
print(confusion_matrix(val_labels, val_predictions))

# Plot learning curves
plt.plot(train_loss_history)
plt.plot(valid_loss_history)
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Plot learning curves
plt.plot(train_acc_history)
plt.plot(valid_acc_history)
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='upper left')
plt.show()